# Декораторы

Для того, чтобы понять как работают декораторы, в первую очередь следует вспомнить, что функции в Python являются объектами, соответственно, функция может принять другую функцию в качестве аргумента, может создать еще одну функцию и может вернуть функцию в качестве возращаемого значения.

Декораторы - "обретки", которые дают возможность изменять поведения функций не меняя их код.

Создадим декоратор:

In [2]:
def decorator(function):
    print('Работает перед оберткой при вызове декоратора')
    def wrapper():
        print('Работает каждый раз перед функцией до ее вызова')
        ret = function()
        print('Работает каждый раз после вызова функции')
        return ret
    print('Работает после обертки  при вызове декоратора')
    return wrapper

Мы создали простой декоратор. Сам декоратор является функцией, которая принимает в качестве аргумента функцию для декорирования. Далее создается "обертка" (функция `wrapper`). По сути, декорируемая функция подменяется оберткой. Каждый раз при вызове декорируемой функции будет срабатывать обертка.

Есть два способа декорировать функцию:

In [3]:
# 1 способ (после создания функции)

def foo():
    print('Работа самой функции')
    
foo = decorator(foo)
print(foo.__name__)

Работает перед оберткой при вызове декоратора
Работает после обертки  при вызове декоратора
wrapper


In [4]:
# 2 способ (сразу при создании)

@decorator
def foo():
    print('Работа самой функции')
    
print(foo.__name__)

Работает перед оберткой при вызове декоратора
Работает после обертки  при вызове декоратора
wrapper


Заметим, что после декорирования функция `foo` называется `wrapper`, то есть мы подменили нашу функцию. Но сама функция никуда не делась, она будт прекрасно работать при вызове в "обретке".

In [5]:
foo()

Работает каждый раз перед функцией до ее вызова
Работа самой функции
Работает каждый раз после вызова функции


Функцию можно декорировать несколькими декораторами.

In [6]:
def decor1(func):
    def wrapper1():
        print('-dec1'.ljust(20, '-'))
        ret = func()
        print('-dec1'.ljust(20, '-'))
        return ret
    return wrapper1

def decor2(func):
    def wrapper2():
        print('dec2~'.rjust(20, '~'))
        ret = func()
        print('dec2~'.rjust(20, '~'))
        return ret
    return wrapper2

@decor1
@decor2
def func():
    print('func'.center(20, '*'))
    
func()

-dec1---------------
~~~~~~~~~~~~~~~dec2~
********func********
~~~~~~~~~~~~~~~dec2~
-dec1---------------


Видим, что сначала функция оборачивается в декоратор, который ближе к объявлению функции. 

Поменяем местами:

In [7]:
@decor2
@decor1
def func():
    print('func'.center(20, '*'))
    
func()

~~~~~~~~~~~~~~~dec2~
-dec1---------------
********func********
-dec1---------------
~~~~~~~~~~~~~~~dec2~


Теперь декоратор 1 оборачивает функцию первый как и ожидалось.

## Передача декораторам аргументов функций

Однако, все декораторы, которые мы рассматривали, не имели одного очень важного функционала — передачи аргументов декорируемой функции. Собственно, это тоже несложно сделать.

In [8]:
def decorator(function):
    
    def wrapper(*args, **kwargs):
        print('Перед вызовом функции имею', args, kwargs)
        ret = function(*args, **kwargs)
        print('После вызова могу менять возращамое значение:', ret)
        return ret
    return wrapper

@decorator
def foo(arg1, arg2, key1='value1', key2=1000):
    print('В самой функции:', arg1, arg2, key1, key2)
    return 7000

In [10]:
foo(100, 200, key2=500)

Перед вызовом функции имею (100, 200) {'key2': 500}
В самой функции: 100 200 value1 500
После вызова могу менять возращамое значение: 7000


7000

Собственно, одно из применений декораторов - этот изменение или проверка передаваемых аргуеметов функции или возвращаемого значения.

Давайте попробуем сделать декоратор для втроенной функции `sum`, где будем умножать каждый аргумент на 2 и возращаемое значение умножим на 2.

In [12]:
def sumdec(func):
    
    def wrapper(nums):
        for i in range(len(nums)):
            nums[i] *= 2
        return func(nums) * 2
    
    return wrapper

In [14]:
sumd = sumdec(sum)

result = sumd([1, 2, 3]) # (1*2 + 2*2 + 3*2) * 2 = 24
print(result)

24


## Декорирование методов

При декорировании методов классов, главное помнить, что первым аргументов у методов всегода идет `self`.

In [16]:
def food_recorder(func):
    """Декоратор учитывающий съеденную еду."""
    def wrapper(self, food):
        wrapper.food_records.append(food)
        func(self, food)
        print(f'People ate in general: {wrapper.food_records}.' )
    wrapper.food_records = []
    return wrapper


class Person:
    
    def __init__(self, name):
        self.name = name

    @food_recorder
    def eat(self, food):
        print(f'{self.name} ate {food}')
        
shia = Person('Shia')
rafael = Person('Rafael')
shia.eat('Apple')
shia.eat('Fish')
rafael.eat('Orange')
shia.eat('Steak')

Shia ate Apple
People ate in general: ['Apple'].
Shia ate Fish
People ate in general: ['Apple', 'Fish'].
Rafael ate Orange
People ate in general: ['Apple', 'Fish', 'Orange'].
Shia ate Steak
People ate in general: ['Apple', 'Fish', 'Orange', 'Steak'].


## Декораторы с аргументами

А теперь попробуем написать декоратор, принимающий аргументы:

In [33]:
def decorator_factory():
    print('Я создаю декораторы. Я буду вызван только раз:'
          ' когда ты попросишь меня создать декоратор.')
    
    def decorator(func):
        print('Я декоратор. Я буду вызван только раз:'
              ' в момент декорирования функции.')
        
        def wrapper(*args, **kwargs):
            print('Я - обёртка вокруг декорируемой функции.\n'
                  'Я буду вызвана каждый раз, когда ты вызываешь'
                  ' декорируемую функцию.')
            
            print('Я вызываю декорируемую функцию:')
            return func(*args, **kwargs)
        print('Я возвращаю обёрнутую функцию.')
        return wrapper
    print('Я возвращаю декоратор.')
    return decorator

In [34]:
# Создаем декоратор
new_decorator = decorator_factory()

Я создаю декораторы. Я буду вызван только раз: когда ты попросишь меня создать декоратор.
Я возвращаю декоратор.


In [35]:
# Декорируем функцию

def foo():
    print('--А я декорируемая функция!--')
    
foo = new_decorator(foo)

Я декоратор. Я буду вызван только раз: в момент декорирования функции.
Я возвращаю обёрнутую функцию.


In [36]:
# Запускаем функцию

foo()

Я - обёртка вокруг декорируемой функции.
Я буду вызвана каждый раз, когда ты вызываешь декорируемую функцию.
Я вызываю декорируемую функцию:
--А я декорируемая функция!--


Второй способ декорирования:

In [37]:
@decorator_factory() # decorator_factory возвращает декоратор, который обертывает foo
def foo():
    print('А я декорируемая фнкция!')

Я создаю декораторы. Я буду вызван только раз: когда ты попросишь меня создать декоратор.
Я возвращаю декоратор.
Я декоратор. Я буду вызван только раз: в момент декорирования функции.
Я возвращаю обёрнутую функцию.


In [38]:
foo()

Я - обёртка вокруг декорируемой функции.
Я буду вызвана каждый раз, когда ты вызываешь декорируемую функцию.
Я вызываю декорируемую функцию:
А я декорируемая фнкция!


Вернёмся к аргументам декораторов, ведь, если мы используем функцию, чтобы создавать декораторы "на лету", мы можем передавать декоратору любые аргументы, как обычной функции. Мы можем использовать и распаковку через \*args и \*\*kwargs в случае необходимости.

In [40]:
def decorator_factory(*dargs, **dkwargs):
    print('Я создаю декораторы. И у меня есть аргументы:\n',
             dargs, '\n', dkwargs)
    
    def decorator(func):
        print('Я декоратор. И могу читать аргументы создателя:\n',
                  dargs, '\n', dkwargs)
        
        def wrapper(*fargs, **fkwargs):
            print('Я - обёртка вокруг декорируемой функции.\n'
                  'И я имею доступ ко всем аргументам.\n'
                  'К аргументам создателя декоратора:\n',
                   dargs, '\n', dkwargs, '\n',
                   'И к аргументам вызываемой функции:\n',
                   fargs, '\n', fkwargs)            
            return func(*fargs, **fkwargs)
        return wrapper
    return decorator


@decorator_factory(10, 20, key1='value1', key2='value2')
def foo(*args, **kwargs):
    print('Я декорируемая функция, и знаю только свои аргументы:\n',
             args, '\n', kwargs)

Я создаю декораторы. И у меня есть аргументы:
 (10, 20) 
 {'key1': 'value1', 'key2': 'value2'}
Я декоратор. И могу читать аргументы создателя:
 (10, 20) 
 {'key1': 'value1', 'key2': 'value2'}


In [41]:
foo(3.14, 9.8, shape='Sphere')

Я - обёртка вокруг декорируемой функции.
И я имею доступ ко всем аргументам.
К аргументам создателя декоратора:
 (10, 20) 
 {'key1': 'value1', 'key2': 'value2'} 
 И к аргументам вызываемой функции:
 (3.14, 9.8) 
 {'shape': 'Sphere'}
Я декорируемая функция, и знаю только свои аргументы:
 (3.14, 9.8) 
 {'shape': 'Sphere'}


## Некоторые особенности работы с декораторами

* Декораторы несколько замедляют вызов функции, не забывайте об этом.
* Вы не можете "раздекорировать" функцию. Безусловно, существуют трюки, позволяющие создать декоратор, который можно отсоединить от функции, но это плохая практика. Правильнее будет запомнить, что если функция декорирована — это не отменить.
* Декораторы оборачивают функции, что может затруднить отладку.

Последняя проблема частично решена добавлением в модуле `functools` функции `functools.wraps`, копирующей всю информацию об оборачиваемой функции (её имя, из какого она модуля, её документацию и т.п.) в функцию-обёртку.

`functools.wraps` тоже является декоратором.

Рассмотрим пример декоратора, который считает время работы функции:

In [45]:
import time
import functools

def timer(func):
    @functools.wraps(func)    # Попробуйте закомментировать эту строчку
                              # и посмотреть как измениться имя функции
    def wrapper():
        tic = time.perf_counter()
        func()
        toc = time.perf_counter()
        print('Время работы:', toc-tic)
    return wrapper
  
    
@timer
def foo():
    for i in range(4000):
        i**i


foo()

# Проверим имя функции
print('Имя функции:', foo.__name__)

Время работы: 0.5484252918024985
Имя функции: foo


Декораторы могут быть использованы для расширения возможностей функций из сторонних библиотек (код которых мы не можем изменять), или для упрощения отладки (мы не хотим изменять код, который ещё не устоялся).

Также полезно использовать декораторы для расширения различных функций одним и тем же кодом, без повторного его переписывания каждый раз.